
# 📘 XGBoost Regression
---

# 1️⃣ Problem Setup

We are solving a **regression problem using XGBoost**.

### Dataset
| CGPA | Package (LPA) |
|------|--------------|
| 6.7  | 4.5 |
| 9.0  | 11 |
| 7.5  | 6 |
| 5.0  | 8 |

Goal:
> Predict package using CGPA

---

# 2️⃣ Big Idea

XGBoost = Gradient Boosting + Improvements

So regression flow is SAME as Gradient Boosting:
1. Start with base model (mean)
2. Compute residuals
3. Train tree on residuals
4. Add prediction
5. Repeat

BUT:
> Trees are DIFFERENT (special XGBoost trees)

---

# 3️⃣ Stage 1 — Base Model

Base estimator = **Mean of targets**

Mean = (4.5 + 11 + 6 + 8) / 4 = **7.3**

So prediction for ALL students:
```
ŷ = 7.3
```

---

# 4️⃣ Residuals (Pseudo-residuals)

Residual = Actual − Prediction

| CGPA | Actual | Pred | Residual |
|------|--------|------|---------|
| 6.7 | 4.5 | 7.3 | -2.8 |
| 9.0 | 11 | 7.3 | 3.7 |
| 7.5 | 6 | 7.3 | -1.3 |
| 5.0 | 8 | 7.3 | 0.7 |

---

# 5️⃣ Build First XGBoost Tree

We now train tree on:
```
Input: CGPA
Target: Residuals
```

---

# 6️⃣ XGBoost Special Concepts

## 6.1 Similarity Score (Node Score)

Equivalent of:
- Gini (classification)
- Variance (regression)

### Formula
```
Similarity Score = (Sum of residuals)² / (Number of residuals + λ)
```

Where:
- λ = regularization parameter
- Assume λ = 0 (simplification)

---

# 7️⃣ Root Node Score

Residuals:
```
[-2.8, 3.7, -1.3, 0.7]
```

Sum = 0.3

Similarity:
```
(0.3²)/4 = 0.02
```

This is parent score.

---

# 8️⃣ Find Split Points

Sort CGPA:
```
5.0, 6.7, 7.5, 9.0
```

Candidate splits:
```
5.85, 7.1, 8.25
```

We test ALL splits.

---

# 9️⃣ Split 1: CGPA < 5.85

Left:
```
[0.7]
```
Right:
```
[-2.8, -1.3, 3.7]
```

### Scores

Left:
```
0.7² / 1 = 0.49
```

Right:
```
(-2.8 -1.3 + 3.7)² / 3 = 0.05
```

### Gain
```
0.49 + 0.05 − 0.02 = 0.52
```

---

# 🔟 Split 2: CGPA < 7.1

Left:
```
[0.7, -2.8]
```
Right:
```
[-1.3, 3.7]
```

Scores:
```
Left = 2.20
Right = 2.88
```

Gain:
```
2.20 + 2.88 − 0.02 = 5.06
```

---

# 1️⃣1️⃣ Split 3: CGPA < 8.25

Left:
```
[0.7, -2.8, -1.3]
```
Right:
```
[3.7]
```

Scores:
```
Left = 3.85
Right = 13.69
```

Gain:
```
3.85 + 13.69 − 0.02 ≈ 17.52
```

✅ Best split = **8.25**

---

# 1️⃣2️⃣ Second Level Split

We now split LEFT node further.

Remaining splits:
- 5.85
- 7.1

After gain calculation:
> Best second split = 5.85

Final tree structure:

```
          CGPA < 8.25
          /            CGPA < 5.85      3.7
     /          7     -2.05
```

---

# 1️⃣3️⃣ Leaf Output Formula

### Leaf Value
```
Output = Sum of residuals / (Number of residuals + λ)
```

With λ = 0 → average residual

---

# 1️⃣4️⃣ Leaf Outputs

| Leaf Residuals | Output |
|---------------|--------|
| [0.7] | 0.7 |
| [-2.8, -1.3] | -2.05 |
| [3.7] | 3.7 |

---

# 1️⃣5️⃣ Stage 2 Model

Combined model:
```
Prediction = Mean + η × Tree Output
```

Where:
- η = learning rate
- Assume η = 0.3

---

# 1️⃣6️⃣ Predictions

Example: CGPA = 6.7

Falls into:
```
CGPA < 8.25 ✔
CGPA < 5.85 ❌ → leaf = -2.05
```

Prediction:
```
7.3 + 0.3 × (-2.05) = 6.69
```

---

# 1️⃣7️⃣ Residuals After Stage 2

| Point | New Residual |
|------|-------------|
| 1 | -2.19 |
| 2 | 2.59 |
| 3 | -0.69 |
| 4 | 0.49 |

Residuals moved closer to **0** ✅

---

# 1️⃣8️⃣ Stage 3+

Repeat:
1. Train new tree on new residuals
2. Add to model

Final model:
```
F(x) = Mean + ηT1 + ηT2 + ηT3 ...
```

---

# 1️⃣9️⃣ Exact vs Approx Algorithm

### Exact Greedy
- Try all splits
- Accurate
- Slow
- Used for small datasets

### Approximate (Histogram-based)
- Bin values
- Faster
- Used in real XGBoost

---

# 2️⃣0️⃣ Important Formulas

## Similarity Score
```
(Sum residuals)² / (n + λ)
```

## Gain
```
LeftScore + RightScore − ParentScore
```

## Leaf Output
```
Sum residuals / (n + λ)
```

## Final Model
```
F(x) = Base + Σ (learning_rate × tree_output)
```

---

# 2️⃣1️⃣ Key Insights

✅ XGBoost = Gradient boosting with:
- Regularization
- Smart splits
- Gain-based trees

✅ Trees predict:
> Residuals, NOT labels

✅ Residuals → 0 ⇒ Perfect model

---